# Movie recommandation system

## Exploring Dataset

In [3]:
import pandas as pd

df = pd.read_csv("../dataset/movies.csv")
df.head()

,id,title,description,release_date,rating,vote_count,genres,actors,director
0,27205,Inception,"Cobb, a skilled thief who commits corporate es...",2010-07-15,8.368,35811,"Action, Science Fiction, Adventure","Leonardo DiCaprio, Joseph Gordon-Levitt, Ken W...",Christopher Nolan
1,157336,Interstellar,The adventures of a group of explorers who mak...,2014-11-05,8.434,34465,"Adventure, Drama, Science Fiction","Matthew McConaughey, Anne Hathaway, Michael Ca...",Christopher Nolan
2,155,The Dark Knight,Batman raises the stakes in his war on crime. ...,2008-07-16,8.515,32012,"Drama, Action, Crime, Thriller","Christian Bale, Heath Ledger, Michael Caine, G...",Christopher Nolan
3,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...",2009-12-15,7.581,30907,"Action, Adventure, Fantasy, Science Fiction","Sam Worthington, Zoe Saldaña, Sigourney Weaver...",James Cameron
4,24428,The Avengers,When an unexpected enemy emerges and threatens...,2012-04-25,7.714,30090,"Science Fiction, Action, Adventure","Robert Downey Jr., Chris Evans, Mark Ruffalo, ...",Joss Whedon


In [4]:
df.shape

(10000, 9)

In [5]:
df.columns

Index(['id', 'title', 'description', 'release_date', 'rating', 'vote_count',
       'genres', 'actors', 'director'],
      dtype='object')

# Preprocessing

In [6]:
def combine_fields(row):
    return (
        f"{row['title']} directed by {row['director']}. "
        f"Genre: {row['genres']}. "
        f"Starring: {row['actors']}. "
        f"{row['description']}"
    )

df['full_text'] = df.apply(combine_fields, axis=1)

# Embeddings

In [7]:
from sentence_transformers import SentenceTransformer


/home/sudarshan/Desktop/Workshop/git_repo/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
model = SentenceTransformer('all-MiniLM-L6-v2')
desc_vector = model.encode(df['full_text'])

desc_vector.shape

(10000, 384)

# Model Training

In [9]:
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(n_neighbors=10, metric='cosine')
knn.fit(desc_vector)


,n_neighbors,10
,radius,1.0
,algorithm,'auto'
,leaf_size,30
,metric,'cosine'
,p,2
,metric_params,None
,n_jobs,None


In [10]:
def recommend_movies(index, no_of_movies=5):
    movie_row = df.iloc[index]
    print(f"Selected Movie: {movie_row['title']}")
    query_vector = model.encode([movie_row['full_text']])
    distances, indices = knn.kneighbors(query_vector, n_neighbors=no_of_movies+1)  
    recommended_df = df.iloc[indices[0][1:]]  

    return recommended_df

recommend_movies(320, 10)

Selected Movie: Godzilla vs. Kong


,id,title,description,release_date,rating,vote_count,genres,actors,director,full_text
1913,823464,Godzilla x Kong: The New Empire,"Following their explosive showdown, Godzilla a...",2024-03-27,7.234,2463,"Science Fiction, Action, Adventure","Rebecca Hall, Brian Tyree Henry, Dan Stevens, ...",Adam Wingard,Godzilla x Kong: The New Empire directed by Ad...
7113,449574,Godzilla: Planet of the Monsters,A desperate group of refugees attempts to reco...,2017-11-17,6.307,468,"Animation, Action, Science Fiction","Mamoru Miyano, Takahiro Sakurai, Kana Hanazawa...",Hiroyuki Seshita,Godzilla: Planet of the Monsters directed by H...
9134,492719,Godzilla: City on the Edge of Battle,Humanity's desperate battle to reclaim the Ear...,2018-05-18,6.500,321,"Animation, Science Fiction","Mamoru Miyano, Yuki Kaji, Tomokazu Sugita, Tak...",Kobun Shizuno,Godzilla: City on the Edge of Battle directed ...
9173,536115,Godzilla: The Planet Eater,"With no means for defeating Godzilla Earth, ma...",2018-11-09,6.437,319,"Animation, Science Fiction, Action, Adventure","Mamoru Miyano, Takahiro Sakurai, Kana Hanazawa...",Hiroyuki Seshita,Godzilla: The Planet Eater directed by Hiroyuk...
704,373571,Godzilla: King of the Monsters,Follows the heroic efforts of the crypto-zoolo...,2019-05-29,6.701,5944,"Science Fiction, Action","Kyle Chandler, Vera Farmiga, Millie Bobby Brow...",Michael Dougherty,Godzilla: King of the Monsters directed by Mic...
3186,940721,Godzilla Minus One,"In postwar Japan, Godzilla brings new devastat...",2023-11-03,7.675,1419,"Science Fiction, Horror, Action","Ryunosuke Kamiki, Minami Hamabe, Yuki Yamada, ...",Takashi Yamazaki,Godzilla Minus One directed by Takashi Yamazak...
375,124905,Godzilla,"Ford Brody, a Navy bomb expert, has just reuni...",2014-05-14,6.334,8871,"Action, Drama, Science Fiction","Aaron Taylor-Johnson, Elizabeth Olsen, Juliett...",Gareth Edwards,Godzilla directed by Gareth Edwards. Genre: Ac...
9943,15767,Godzilla: Final Wars,"Humanity finally rids themselves of Godzilla, ...",2004-12-04,7.000,283,"Action, Adventure, Science Fiction, Fantasy, T...","Masahiro Matsuoka, Rei Kikukawa, Don Frye, Mak...",Ryûhei Kitamura,Godzilla: Final Wars directed by Ryûhei Kitamu...
4472,1678,Godzilla,Japan is thrown into a panic after several shi...,1954-11-03,7.605,911,"Thriller, Horror, Science Fiction","Akira Takarada, Momoko Kôchi, Akihiko Hirata, ...",Ishirō Honda,Godzilla directed by Ishirō Honda. Genre: Thri...
1244,929,Godzilla,French nuclear tests irradiate an iguana into ...,1998-05-20,5.616,3762,"Science Fiction, Action, Thriller","Matthew Broderick, Jean Reno, Maria Pitillo, H...",Roland Emmerich,Godzilla directed by Roland Emmerich. Genre: S...


In [11]:
def recommend_movies_by_text(text, no_of_movies=5):
    query_vector = model.encode([text])
    distances, indices = knn.kneighbors(query_vector, n_neighbors=no_of_movies)  
    recommended_df = df.iloc[indices[0]]  

    return recommended_df

recommend_movies_by_text("The movie about spider", 10)

,id,title,description,release_date,rating,vote_count,genres,actors,director,full_text
4923,9613,Spider,A mentally disturbed man takes residence in a ...,2002-11-06,6.666,800,"Drama, Mystery, Thriller","Ralph Fiennes, Miranda Richardson, Gabriel Byr...",David Cronenberg,Spider directed by David Cronenberg. Genre: Dr...
3343,6488,Arachnophobia,A large spider from the jungles of South Ameri...,1990-07-20,6.384,1322,"Comedy, Horror","Jeff Daniels, Harley Jane Kozak, John Goodman,...",Frank Marshall,Arachnophobia directed by Frank Marshall. Genr...
124,324857,Spider-Man: Into the Spider-Verse,Struggling to find his place in the world whil...,2018-12-06,8.404,15055,"Animation, Action, Adventure, Science Fiction","Shameik Moore, Jake Johnson, Hailee Steinfeld,...",Bob Persichetti,Spider-Man: Into the Spider-Verse directed by ...
66,557,Spider-Man,After being bitten by a genetically altered sp...,2002-05-01,7.296,18468,"Action, Science Fiction","Tobey Maguire, Willem Dafoe, Kirsten Dunst, Ja...",Sam Raimi,Spider-Man directed by Sam Raimi. Genre: Actio...
639,569094,Spider-Man: Across the Spider-Verse,"After reuniting with Gwen Stacy, Brooklyn’s fu...",2023-05-31,8.362,6329,"Animation, Action, Adventure, Science Fiction","Shameik Moore, Hailee Steinfeld, Brian Tyree H...",Joaquim Dos Santos,Spider-Man: Across the Spider-Verse directed b...
133,558,Spider-Man 2,Peter Parker is going through a major identity...,2004-06-25,7.274,14695,"Action, Adventure, Science Fiction","Tobey Maguire, Kirsten Dunst, James Franco, Al...",Sam Raimi,Spider-Man 2 directed by Sam Raimi. Genre: Act...
3061,615469,Spiderhead,A prisoner in a state-of-the-art penitentiary ...,2022-06-17,5.795,1459,"Science Fiction, Thriller","Chris Hemsworth, Miles Teller, Jurnee Smollett...",Joseph Kosinski,Spiderhead directed by Joseph Kosinski. Genre:...
175,102382,The Amazing Spider-Man 2,"For Peter Parker, life is busy. Between taking...",2014-04-16,6.500,12858,"Action, Adventure, Science Fiction","Andrew Garfield, Emma Stone, Jamie Foxx, Dane ...",Marc Webb,The Amazing Spider-Man 2 directed by Marc Webb...
120,429617,Spider-Man: Far From Home,Peter Parker and his friends go on a summer tr...,2019-06-28,7.400,15199,"Action, Adventure, Science Fiction","Tom Holland, Samuel L. Jackson, Jake Gyllenhaa...",Jon Watts,Spider-Man: Far From Home directed by Jon Watt...
86,1930,The Amazing Spider-Man,Peter Parker is an outcast high schooler aband...,2012-06-23,6.701,16923,"Action, Adventure, Science Fiction","Andrew Garfield, Emma Stone, Rhys Ifans, Marti...",Marc Webb,The Amazing Spider-Man directed by Marc Webb. ...


# Saving Our Model

In [ ]:
import joblib

joblib.dump(knn, "../movie_project/knn.pkl")
joblib.dump(model, "../movie_project/embedding_model.pkl")

['embedding_model.pkl']